In [1]:
import numpy as np
import os

from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')




c:\Users\georg\anaconda3\envs\band\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#!unzip dataset_5.zip

#label_map = {1: 'black', 2:'brown',3:'red',4:'orange',5:'yellow',6:'green',7:'blue',8:'violet',9:'gray',10:'white',11:'gold',12:'silver'}
label_map = {1:'band'}







train_images_dir = 'dataset_5/train/images'
train_annotations_dir = 'dataset_5/train/annotations'
train_data = object_detector.DataLoader.from_pascal_voc(train_images_dir, train_annotations_dir, label_map=label_map)


val_images_dir = 'dataset_5/validation/images'
val_annotations_dir = 'dataset_5/validation/annotations'
validation_data = object_detector.DataLoader.from_pascal_voc(val_images_dir, val_annotations_dir, label_map=label_map)

test_images_dir = 'dataset_5/test/images'
test_annotations_dir = 'dataset_5/test/annotations'
test_data = object_detector.DataLoader.from_pascal_voc(test_images_dir, test_annotations_dir, label_map=label_map)

INFO:tensorflow:Cache will be stored in C:\Users\georg\AppData\Local\Temp\tmpj_n4vd77 with prefix filename fd2c112752054ea1fcebcf49d7779205. Cache_prefix is C:\Users\georg\AppData\Local\Temp\tmpj_n4vd77\fd2c112752054ea1fcebcf49d7779205
INFO:tensorflow:On image 0
INFO:tensorflow:On image 100
INFO:tensorflow:On image 200
INFO:tensorflow:On image 300
INFO:tensorflow:On image 400
INFO:tensorflow:On image 500
INFO:tensorflow:On image 600
INFO:tensorflow:Cache will be stored in C:\Users\georg\AppData\Local\Temp\tmpyz0uzpvx with prefix filename fd2c112752054ea1fcebcf49d7779205. Cache_prefix is C:\Users\georg\AppData\Local\Temp\tmpyz0uzpvx\fd2c112752054ea1fcebcf49d7779205
INFO:tensorflow:On image 0
INFO:tensorflow:Cache will be stored in C:\Users\georg\AppData\Local\Temp\tmp6_tekvnh with prefix filename fd2c112752054ea1fcebcf49d7779205. Cache_prefix is C:\Users\georg\AppData\Local\Temp\tmp6_tekvnh\fd2c112752054ea1fcebcf49d7779205


In [3]:
spec = object_detector.EfficientDetLite1Spec()

In [4]:
model = object_detector.create(train_data=train_data, model_spec=spec, validation_data=validation_data, epochs=30, batch_size=8, train_whole_model=True)

INFO:tensorflow:Retraining the models...


OSError: SavedModel file does not exist at: C:\Users\georg\AppData\Local\Temp\tfhub_modules\924b1ab4d4a5b2c6d6709a01993a9c4bf23f7dd7\{saved_model.pbtxt|saved_model.pb}

In [ ]:
model.evaluate(validation_data)

In [ ]:
TFLITE_FILENAME = 'laptop1.tflite'
LABELS_FILENAME = 'band-labels.txt'

In [ ]:

from tflite_model_maker.config import ExportFormat
model.export(export_dir='.', tflite_filename=TFLITE_FILENAME, label_filename=LABELS_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

In [ ]:
model.evaluate_tflite(TFLITE_FILENAME, test_data)

In [ ]:
#PART 2, USING LITE MODEL
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=3)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file(LABELS_FILENAME)
interpreter = tflite.Interpreter(TFLITE_FILENAME)
interpreter.allocate_tensors()

# Resize the image for input
image = Image.open('dataset/Resistor_+0.jpg')
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.4, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)
image